In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## Empezamos con el Analisis 

In [2]:
url = "../data/raw/marketing_data_simulated.csv"
df = pd.read_csv(url)

In [3]:
df

,Date,Action_Type,Language,Content_Category,Day_of_Week,Impressions,CTR,Clicks
0,2023-01-02,Instagram Post,English,Partners,Monday,1759,0.016391,29
1,2023-01-02,LinkedIn Post,Spanish,Events,Monday,1368,0.008438,12
2,2023-01-03,Instagram Post,Spanish,Humor,Tuesday,1573,0.016354,26
3,2023-01-03,LinkedIn Post,English,Events,Tuesday,1516,0.007464,11
4,2023-01-04,Instagram Post,Spanish,Partners,Wednesday,1821,0.013398,24
...,...,...,...,...,...,...,...,...
1111,2024-12-27,LinkedIn Post,Spanish,Platform Improvements,Friday,454,0.003422,2
1112,2024-12-30,Instagram Post,English,Humor,Monday,1153,0.013793,16
1113,2024-12-30,LinkedIn Post,English,Events,Monday,849,0.004059,3
1114,2024-12-31,Instagram Post,English,Events,Tuesday,1338,0.007941,11


In [4]:
df.head()

,Date,Action_Type,Language,Content_Category,Day_of_Week,Impressions,CTR,Clicks
0,2023-01-02,Instagram Post,English,Partners,Monday,1759,0.016391,29
1,2023-01-02,LinkedIn Post,Spanish,Events,Monday,1368,0.008438,12
2,2023-01-03,Instagram Post,Spanish,Humor,Tuesday,1573,0.016354,26
3,2023-01-03,LinkedIn Post,English,Events,Tuesday,1516,0.007464,11
4,2023-01-04,Instagram Post,Spanish,Partners,Wednesday,1821,0.013398,24


In [5]:
df.tail()

,Date,Action_Type,Language,Content_Category,Day_of_Week,Impressions,CTR,Clicks
1111,2024-12-27,LinkedIn Post,Spanish,Platform Improvements,Friday,454,0.003422,2
1112,2024-12-30,Instagram Post,English,Humor,Monday,1153,0.013793,16
1113,2024-12-30,LinkedIn Post,English,Events,Monday,849,0.004059,3
1114,2024-12-31,Instagram Post,English,Events,Tuesday,1338,0.007941,11
1115,2024-12-31,LinkedIn Post,English,Platform Improvements,Tuesday,953,0.007764,7


In [6]:
df.shape

(1116, 8)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116 entries, 0 to 1115
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              1116 non-null   object 
 1   Action_Type       1116 non-null   object 
 2   Language          1116 non-null   object 
 3   Content_Category  1116 non-null   object 
 4   Day_of_Week       1116 non-null   object 
 5   Impressions       1116 non-null   int64  
 6   CTR               1116 non-null   float64
 7   Clicks            1116 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 69.9+ KB


## Verificamos que no hay Nulos

In [8]:
df.isnull().sum()

Date                0
Action_Type         0
Language            0
Content_Category    0
Day_of_Week         0
Impressions         0
CTR                 0
Clicks              0
dtype: int64

## Empezar transformacion  de datos 

In [12]:
import re

# Read the raw data
with open('../data/raw/marketing_data_simulated.csv', 'r') as file:
    lines = file.readlines()
data = []
for line in lines:
    match = re.match(r'([^,]+),"([^"]+)","([^"]+)","([^"]+)","([^"]+)",(\d+),([0-9.]+),(\d+)', line.strip())
    if match:
        date, action_type, language, content_category, day_of_week, impressions, ctr, clicks = match.groups()
        data.append({
            'Date': date,
            'Action_Type': action_type,
            'Language': language,
            'Content_Category': content_category,
            'Day_of_Week': day_of_week,
            'Impressions': int(impressions),
            'CTR': float(ctr),
            'Clicks': int(clicks)
        })

In [13]:
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Month_Year'] = df['Date'].dt.to_period('M')

## Analisis descriptivo

In [14]:
for col in df.select_dtypes(include=['object']).columns:
    if col != 'Date':
        print(f"{col}: {df[col].nunique()} valores únicos")
        print(f"  Valores: {df[col].unique()}")
        print()

Action_Type: 3 valores únicos
  Valores: ['Instagram Post' 'LinkedIn Post' 'Newsletter']

Language: 7 valores únicos
  Valores: ['English' 'Spanish' 'French' 'Italian' 'Spanish (Spain)'
 'Spanish (LatAm)' 'Portuguese']

Content_Category: 6 valores únicos
  Valores: ['Partners' 'Events' 'Humor' 'Marketing Automation'
 'Platform Improvements' 'Monthly Newsletter']

Day_of_Week: 7 valores únicos
  Valores: ['Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Sunday' 'Saturday']



## Canal de distribución

In [15]:
print(df['Action_Type'].value_counts())
print(f"\nPorcentajes:")
print(df['Action_Type'].value_counts(normalize=True) * 100)

Action_Type
Instagram Post    522
LinkedIn Post     522
Newsletter         72
Name: count, dtype: int64

Porcentajes:
Action_Type
Instagram Post    46.774194
LinkedIn Post     46.774194
Newsletter         6.451613
Name: proportion, dtype: float64


## Estadísticas descriptivas

In [16]:
df.describe()

,Date,Impressions,CTR,Clicks,Month,Year
count,1116,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000
mean,2023-12-31 19:21:17.419354880,2469.619176,0.020529,105.397849,6.501792,2023.501792
min,2023-01-02 00:00:00,155.000000,0.000975,0.000000,1.000000,2023.000000
25%,2023-07-03 00:00:00,1509.750000,0.010522,16.000000,4.000000,2023.000000
50%,2024-01-01 12:00:00,1850.500000,0.014112,25.000000,7.000000,2024.000000
75%,2024-07-02 00:00:00,2303.000000,0.018480,42.000000,10.000000,2024.000000
max,2024-12-31 00:00:00,17357.000000,0.157108,2512.000000,12.000000,2024.000000
std,NaN,2369.047376,0.023838,304.042624,3.449442,0.500221


In [17]:
print("\n--- Estadísticas descriptivas de métricas ---")
print(df[['Impressions', 'CTR', 'Clicks']].describe())


--- Estadísticas descriptivas de métricas ---
        Impressions          CTR       Clicks
count   1116.000000  1116.000000  1116.000000
mean    2469.619176     0.020529   105.397849
std     2369.047376     0.023838   304.042624
min      155.000000     0.000975     0.000000
25%     1509.750000     0.010522    16.000000
50%     1850.500000     0.014112    25.000000
75%     2303.000000     0.018480    42.000000
max    17357.000000     0.157108  2512.000000


In [22]:
print("\n📅 Rango de fechas:")
print(f"Desde: {df['Date'].min().strftime('%Y-%m-%d')}")
print(f"Hasta: {df['Date'].max().strftime('%Y-%m-%d')}")
print(f"Duración: {(df['Date'].max() - df['Date'].min()).days} días")


📅 Rango de fechas:
Desde: 2023-01-02
Hasta: 2024-12-31
Duración: 729 días


## Valores unicos por colummna categorica

In [23]:
categorical_cols = ['Action_Type', 'Language', 'Content_Category', 'Day_of_Week']
for col in categorical_cols:
    print(f"\n{col}:")
    value_counts = df[col].value_counts()
    for value, count in value_counts.items():
        percentage = (count / len(df)) * 100
        print(f"  {value}: {count} ({percentage:.1f}%)")


Action_Type:
  Instagram Post: 522 (46.8%)
  LinkedIn Post: 522 (46.8%)
  Newsletter: 72 (6.5%)

Language:
  English: 443 (39.7%)
  Spanish: 424 (38.0%)
  Italian: 74 (6.6%)
  French: 64 (5.7%)
  Portuguese: 63 (5.6%)
  Spanish (Spain): 24 (2.2%)
  Spanish (LatAm): 24 (2.2%)

Content_Category:
  Platform Improvements: 287 (25.7%)
  Marketing Automation: 271 (24.3%)
  Humor: 202 (18.1%)
  Events: 147 (13.2%)
  Partners: 137 (12.3%)
  Monthly Newsletter: 72 (6.5%)

Day_of_Week:
  Monday: 222 (19.9%)
  Wednesday: 220 (19.7%)
  Friday: 220 (19.7%)
  Thursday: 217 (19.4%)
  Tuesday: 216 (19.4%)
  Sunday: 12 (1.1%)
  Saturday: 9 (0.8%)


### Metricas clave

In [24]:
print(f"Total de impresiones: {df['Impressions'].sum():,}")
print(f"Total de clicks: {df['Clicks'].sum():,}")
print(f"CTR promedio: {df['CTR'].mean():.4f} ({df['CTR'].mean()*100:.2f}%)")
print(f"CTR mediano: {df['CTR'].median():.4f} ({df['CTR'].median()*100:.2f}%)")

Total de impresiones: 2,756,095
Total de clicks: 117,624
CTR promedio: 0.0205 (2.05%)
CTR mediano: 0.0141 (1.41%)
